## ISR - Gen 3

In this notebook, we test running ISR with the Gen3 butler\
Craig Lage - 01-Mar-22

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
import lsst.afw.cameraGeom.utils as camGeomUtils
import lsst.afw.display as afwDisplay
from lsst.ip.isr.isrTask import IsrTask, IsrTaskConfig
from lsst.daf.butler import Butler

In [ ]:
butler = Butler('/repo/main', collections="LATISS/raw/all")

In [ ]:
# Gen3 butler
dayObs = '2022-02-16'
dayObs = int(dayObs.replace('-', ''))

exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
for [id,record] in exposureList:
    print(record.id, record.observation_type, record.exposure_time, record.physical_filter, record.target_name)


In [ ]:
# Config for processing raw bias image
isrConfig = IsrTaskConfig()
isrConfig.doLinearize = False
isrConfig.doBias = False
isrConfig.doFlat = False
isrConfig.doDark = False
isrConfig.doFringe = False
isrConfig.doDefect = False
isrConfig.doWrite = False
isrConfig.doBrighterFatter = False
isrTask = IsrTask(config=isrConfig)

In [ ]:
biasExpId = 2022021600015
rawBiasImage = butler.get('raw', detector=0, exposure=biasExpId)

In [ ]:
biasPostISR = isrTask.run(rawBiasImage).exposure

In [ ]:
flatExpId=2022021600060
rawFlatImage = butler.get('raw', detector=0, exposure=flatExpId)


In [ ]:
isrConfig.doBias = True
#isrConfig.doApplyGains = True
isrTask = IsrTask(config=isrConfig)

In [ ]:
flatPostISR = isrTask.run(rawFlatImage, bias=biasPostISR).exposure

In [ ]:
arrayData = flatPostISR.image.array
mean = arrayData.mean()
std = arrayData.std()
print(mean, std)

In [ ]:
# Look at the data with matplotlib
# The raw data doesn't look very good, because of the large pedestal of about 15,000 ADU
from matplotlib.colors import LogNorm
# Now let's look at ithem
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

plt.figure(figsize=(8,8))
plt.suptitle(f"Image",fontsize=18)
arr = flatPostISR.image.array
img = plt.imshow(arr, norm=LogNorm(vmin=mean-3.0*std, vmax=mean+3.0*std), interpolation='Nearest', cmap='gray')
colorbar(img)
plt.tight_layout(h_pad=1)
#plt.savefig(REPO_DIR+"/plots/NGC4755_17Feb21.pdf")